# Scenario modelling

## Set up environment

In [1]:
CM_BASEPATH = '../cibusmod'

import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), CM_BASEPATH))

In [2]:
import CIBUSmod as cm
import CIBUSmod.utils.plot as plot

import time
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import cvxpy

from typing import Literal

In [3]:
from CIBUSmod.utils.misc import inv_dict, aggregate_data_coords_pair
from CIBUSmod.optimisation.indexed_matrix import IndexedMatrix
from CIBUSmod.optimisation.utils import make_cvxpy_constraint
from itertools import product

In [4]:
# Create session
session = cm.Session(
    name = 'main-foo',
    data_path = "data",
    data_path_default = CM_BASEPATH + "/data/default",
)

# Load scenarios
# ==============

session.add_scenario(
    "BASELINE", years=[2020], pars = "all",
    scenario_workbooks="default_fix"
)

session.add_scenario(
    "SCN_CORE", years=[2020], pars = "all",
    scenario_workbooks="base"
)

session.add_scenario(
    "SCN_MIN_LEY", years=[2020], pars = "all",
    scenario_workbooks=["base", "scn-min-ley"]
)

session.add_scenario(
    "SCN_SNG", years=[2020], pars = "all",
    scenario_workbooks="base"
)

session.add_scenario(
    "SCN_ORG", years=[2020], pars = "all",
    scenario_workbooks="base"
)

session.add_scenario(
    "SCN_DEMAND", years=[2020], pars="all",
    scenario_workbooks="scn-demand",
)
   

A scenario with the name 'BASELINE' already exists use .update_scenario() or .remove_scenario() instead.
A scenario with the name 'SCN_CORE' already exists use .update_scenario() or .remove_scenario() instead.
A scenario with the name 'SCN_MIN_LEY' already exists use .update_scenario() or .remove_scenario() instead.
A scenario with the name 'SCN_SNG' already exists use .update_scenario() or .remove_scenario() instead.
A scenario with the name 'SCN_ORG' already exists use .update_scenario() or .remove_scenario() instead.
A scenario with the name 'SCN_DEMAND' already exists use .update_scenario() or .remove_scenario() instead.


In [5]:
%%time

scn = "SCN_DEMAND"

retrievers = {
    'Regions': cm.ParameterRetriever('Regions'),
    'DemandAndConversions': cm.ParameterRetriever('DemandAndConversions'),
    'CropProduction': cm.ParameterRetriever('CropProduction'),
    'FeedMgmt': cm.ParameterRetriever('FeedMgmt'),
    'GeoDistributor': cm.ParameterRetriever('GeoDistributor'),
}
    
cm.ParameterRetriever.update_all_parameter_values(**session[scn], year=2020)

# Instatiate Regions
regions = cm.Regions(
    par = retrievers['Regions'],
)

# Instantiate DemandAndConversions
demand = cm.DemandAndConversions(
    par = retrievers['DemandAndConversions'],
)

# Instantiate CropProduction
crops = cm.CropProduction(
    par = retrievers['CropProduction'],
    index = regions.data_attr.get('x0_crops').index
)

# Instantiate AnimalHerds
# Each AnimalHerd object is stored in an indexed pandas.Series
herds = cm.make_herds(regions, sub_systems={
#    'cattle': ['ley based'], 
#    ('cattle', 'dairy', 'conventional'): ['maize based'], 
#    ('cattle', 'beef', 'conventional'): ['maize based'],
    'sheep': ['autumn lamb', 'spring lamb', 'winter lamb', 'other sheep']
})


# Instantiate feed management
feed_mgmt = cm.FeedMgmt(
    herds = herds,
    par = retrievers['FeedMgmt'],
)

# Instantiate geo distributor
optproblem = cm.FeedDistributor(
    regions = regions,
    demand = demand,
    crops = crops,
    herds = herds,
    feed_mgmt = feed_mgmt,
    par = retrievers['GeoDistributor'],
)

self = optproblem

# Instantiate WasteAndCircularity
waste = cm.WasteAndCircularity(
    demand = demand,
    crops = crops,
    herds = herds,
    par = cm.ParameterRetriever('WasteAndCircularity')
)

# Instantiate by-product management
byprod_mgmt = cm.ByProductMgmt(
    demand = demand,
    herds = herds,
    par = cm.ParameterRetriever('ByProductMgmt')
)

# Instantiate manure management
manure_mgmt = cm.ManureMgmt(
    herds = herds,
    feed_mgmt = feed_mgmt,
    par = cm.ParameterRetriever('ManureMgmt'),
    settings = {
        'NPK_excretion_from_balance' : True
    }
)

# Instantiate crop residue managment
crop_residue_mgmt = cm.CropResidueMgmt(
    demand = demand,
    crops = crops,
    herds = herds,
    par = cm.ParameterRetriever('CropResidueMgmt')
)

# Instantiate plant nutrient management
plant_nutrient_mgmt = cm.PlantNutrientMgmt(
    demand = demand,
    regions = regions,
    crops = crops,
    waste = waste,
    herds = herds,
    par = cm.ParameterRetriever('PlantNutrientMgmt')
)

# Instatiate machinery and energy management
machinery_and_energy_mgmt  = cm.MachineryAndEnergyMgmt(
    regions = regions,
    crops = crops,
    waste = waste,
    herds = herds,
    par = cm.ParameterRetriever('MachineryAndEnergyMgmt')
)

# Instatiate inputs management
inputs = cm.InputsMgmt(
    demand = demand,
    crops = crops,
    waste = waste,
    herds = herds,
    par = cm.ParameterRetriever('InputsMgmt')
)

/root/cibus/scenarios/../cibusmod/CIBUSmod/utils/retriever.py:152: UserWarning: 
-----------------------------------------------------------------------------
Some filter values included in data were not available in relation_tables.xlsx.
Missing for 'by_prod': 'soybean protein concentrate', 'maize gluten meal', 'soybean meal', 'fish meal', 'cream', 'luzern meal', 'palm kernel expeller'
------------------------------------------------------------------------------
  warnings.warn(f"""


CPU times: user 11.5 s, sys: 81.6 ms, total: 11.6 s
Wall time: 11.6 s


In [6]:
cm.ParameterRetriever.update_all_parameter_values()
cm.ParameterRetriever.update_relation_tables()

cm.ParameterRetriever.update_all_parameter_values(**session[scn], year=2020)

regions.calculate()
demand.calculate()
crops.calculate()
for h in herds:
    h.calculate()
    

/root/cibus/scenarios/../cibusmod/CIBUSmod/utils/retriever.py:152: UserWarning: 
-----------------------------------------------------------------------------
Some filter values included in data were not available in relation_tables.xlsx.
Missing for 'crop': 'Lentils'
Missing for 'animal': 'calves'
Missing for 'by_prod': 'soybean protein concentrate', 'maize gluten meal', 'soybean meal', 'fish meal', 'cream', 'luzern meal', 'palm kernel expeller'
------------------------------------------------------------------------------
  warnings.warn(f"""
/root/cibus/scenarios/../cibusmod/CIBUSmod/utils/retriever.py:164: UserWarning: 
--------------------------------------------------------------------------------------------
Data includes crop(s) without an x0 area specified through 'x0_crops' in the Regions module.
Missing: 'Lentils', 'Wheat (add)', 'Peas (add)'
--------------------------------------------------------------------------------------------
  warnings.warn(f"""
/root/cibus/scenario

[ME, fat, rough, PBV, AAT, DM] [ME, fat, rough, PBV, AAT, DM] [ME, fat, rough, PBV, AAT, DM] [ME, fat, rough, PBV, AAT, DM] [ME] [ME] [ME] [ME] [ME] [ME] [ME] [ME] [NE] [NE] [DM] [DM] [DM] [DM] [DM] [DM] [DM] [DM] [DM] [DM] [DM] [DM] 

In [7]:
self.make_x0()
sng_areas = self.x0["crp"].loc[
    [
        "Semi-natural meadows",
        "Semi-natural pastures, thin soils",
        "Semi-natural pastures, wooded", 
        "Semi-natural pastures"
    ]
]
fallow_areas = self.x0["crp"].loc[["Fallow"]]

C8_pars = {
    "C8_crp": [sng_areas, fallow_areas],
    "C8_rel": ["<=",      "=="],
    "C8_tol": [None,      1e-4],
}

cons=[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 14]
if scn in "BASELINE":
    cons.remove(8)
elif "_SNG" in scn:
    tol = 1e-4
    C8_pars["C8_crp"][0] = sng_areas * (1-tol)
    C8_pars["C8_rel"][0] = ">="


self.make(cons, verbose=True, **C8_pars)

[09:40:20][FeedDistributor.make] Getting x0 and making indexes ... 5.7s
[09:40:26][FeedDistributor.make] Creating demand vector ... 0.0s
[09:40:26][FeedDistributor.make] Calculating scaling factors ... 0.0s
[09:40:26][FeedDistributor.make] Making objective O1 ... 0.2s
[09:40:26][FeedDistributor.make] Making constraint C1... 1.2s
[09:40:28][FeedDistributor.make] Making constraint C2... 0.4s
[09:40:28][FeedDistributor.make] Making constraint C3... 0.4s
[09:40:28][FeedDistributor.make] Making constraint C4... 0.3s
[09:40:29][FeedDistributor.make] Making constraint C5... 0.5s
[09:40:29][FeedDistributor.make] Making constraint C6... 0.8s
[09:40:30][FeedDistributor.make] Making constraint C8... 0.0s
[09:40:30][FeedDistributor.make] Making constraint C10... 0.3s
[09:40:30][FeedDistributor.make] Making constraint C11... 25.4s
[09:40:56][FeedDistributor.make] Making constraint C12... 17.4s
[09:41:13][FeedDistributor.make] Making constraint C14... 15.1s
[09:41:28][FeedDistributor.make] Making co

In [8]:
def make_CX_organic_cattle(tol = 0.001):
    cr_x0_org = self.x0["ani"][["cattle"]].xs("organic", level="prod_system")
    cr_x0_con = self.x0["ani"][["cattle"]].xs("conventional", level="prod_system")
    both_zero = cr_x0_org[cr_x0_org == 0].index.intersection(cr_x0_con[cr_x0_con==0].index)
    cr_x0_org = cr_x0_org.reindex(cr_x0_org.index.difference(both_zero))
    
    shares_df = (
        (cr_x0_org / cr_x0_con.reindex(cr_x0_org.index, fill_value=0))
        .replace({np.inf: np.nan, -np.inf: np.nan})
        .fillna(1)
        .to_frame(name="share")
    )
    shares_df *= (1-tol)
    shares_df = shares_df.reset_index()

    col_idx = self.x_idx["ani"]
    row_idx = cr_x0_org.index
    
    row_idx_df = row_idx.to_frame(index=False).reset_index(names="row_i")
    col_idx_df = col_idx.to_frame(index=False).reset_index(names="col_i")
    
    merged = row_idx_df.merge(col_idx_df, on=row_idx.names).merge(shares_df, on=["species", "breed", "region"])
    merged["values"] = np.where(
        merged["prod_system"] == "organic",
        1 - merged["share"],
        -merged["share"]
    )

    n_rows = len(row_idx)
    n_cols = len(col_idx)
    
    row_i = merged["row_i"].to_numpy()
    col_i = merged["col_i"].to_numpy()
    values = merged["values"].to_numpy()

    M = scipy.sparse.hstack([
        scipy.sparse.coo_array((values, (row_i, col_i)), shape=(n_rows, n_cols)).tocsc(),
        scipy.sparse.csc_array((n_rows, len(self.x_idx["crp"]))),
        scipy.sparse.csc_array((n_rows, len(self.x_idx["fds"]))),
    ], format="csc")

    IM = IndexedMatrix(M, row_idx=row_idx, col_idx={})
    
    self.constraints["CX: Share of organic cattle"] = {
        "left": lambda x, A: A.M @ x,
        "right": lambda A: 0,
        "rel": ">=",
        "pars": { "A": IM },
    }
    print("Added constraint ensuring a maintained share of organic cattle production.")
    
if "_ORG" in scn: 
    make_CX_organic_cattle()
    self.make_C7()

## Data debugging

### Print size of matrices (latex table)

In [9]:
for (k, IM) in self.matrices().items():
    M = IM.M
    nrows, ncols = M.shape
    p = 100 * (M.nnz / (nrows*ncols))
    print(" & ".join(map(str, [
        k,
        nrows,
        ncols,
        M.nnz,
        f"{p:.3f}\\%\\\\"
    ])))

OBJ.P1 & 330084 & 328637 & 14453 & 0.000\%\\
C1.A1 & 88 & 328637 & 510701 & 1.766\%\\
C2.A2 & 1272 & 328637 & 62124 & 0.015\%\\
C3.A3 & 318 & 328637 & 35091 & 0.034\%\\
C4.A4 & 8 & 328637 & 22048 & 0.839\%\\
C5.A5 & 212 & 328637 & 10812 & 0.016\%\\
C6_max.A6 & 424 & 328637 & 46788 & 0.034\%\\
C8_0.A8 & 848 & 328637 & 848 & 0.000\%\\
C8_1(low).A8 & 212 & 328637 & 212 & 0.000\%\\
C8_1(upp).A8 & 212 & 328637 & 212 & 0.000\%\\
C10.A10 & 70 & 328637 & 116282 & 0.505\%\\
C11 (min).A11 & 314184 & 328637 & 3324160 & 0.003\%\\
C11 (eq).A11 & 314184 & 328637 & 5196968 & 0.005\%\\
C11 (max).A11 & 314184 & 328637 & 118720 & 0.000\%\\
C12 (min).A12 & 8480 & 328637 & 233200 & 0.008\%\\
C12 (eq).A12 & 13144 & 328637 & 329448 & 0.008\%\\
C12 (max).A12 & 8480 & 328637 & 241680 & 0.009\%\\
C14 (min).A14 & 314184 & 328637 & 3324160 & 0.003\%\\
C14 (max).A14 & 314184 & 328637 & 3324160 & 0.003\%\\


## Improve numerics

In [10]:
def improve_numerics(self):
    for name, C in self.constraints.items():
        M = [obj for obj in C['pars'].values() if isinstance(obj, IndexedMatrix)]
        assert len(M) == 1, "Expected one and only one IndexedMatrix"
        M = M[0]
        max_val = M.M.max()
        if max_val == 1:
            continue
        for name2, obj in C['pars'].items():
            if name2=="tol":
                continue
            try:
                if isinstance(obj, IndexedMatrix):
                    obj.M = obj.M / max_val
                else:
                    obj[:] = obj / max_val
            except Exception as e:
                print(f"Failed building constraint {name}, name2={name2}.")
                raise e

    print("Completed rescaling of matrices.")

improve_numerics(optproblem)

Completed rescaling of matrices.


In [11]:
# =========================================================================== #
# --------------- SOLVE, STORE, AND STOP IF ON BASELINE SCENARIO ------------ #
# =========================================================================== #

if scn == "BASELINE":
    self.solve(
        apply_solution=True,
        verbose=True,
        solver_settings=[{
            "solver": "GUROBI",
            "reoptimize": True,
            "verbose": True,
        }]
    )
    self.feed_mgmt.calculate()
    session.store(
        scn, 2020,
        demand, regions, crops, herds, optproblem
    )
    
    raise Exception("Stopping for BASELINE scenario")

# =========================================================================== #
# ---------------------------- OTHERWISE, CONTINUE -------------------------- #
# =========================================================================== #

# Replace the objective function

While the original optimisation objective focused on minimising the change, we now instead want to maximize the protein contents.

## Mapping `x` to protein contents

First we need to create a row-array that maps each element in `x` with its protein content, so that we compute the aggregate protein amount from the decision variable.

In [12]:
ADDED_PEAS = "Peas (add)"
ADDED_WHEAT = "Wheat (add)"

PROTEIN_CONTENTS = {
    ADDED_PEAS: 220,
    ADDED_WHEAT: 67.15,
    "meat": 155.5,
    "milk": 35.0,
}

# Convert to thousands of prot. / kg, instead of straight
for k in PROTEIN_CONTENTS.keys():
    PROTEIN_CONTENTS[k] /= 1e3

def make_protein_mask_ani():
    RELEVANT_ANIMAL_PRODUCTS = ["meat", "milk"]
    
    # Get row index from animal product demand vector (ps,sp,ap)
    row_idx = pd.MultiIndex.from_tuples(
        [
            ("conventional", "cattle", "meat"),
            ("conventional", "cattle", "milk"),
            ("organic", "cattle", "meat"),
            ("organic", "cattle", "milk"),
        ],
        names=["prod_system", "species", "animal_prod"]
    )

    # Get col index from animal herds (sp,br,ss,ps,re)
    col_idx = self.x_idx_short["ani"]

    # To store data and corresponding row/col numbers for constructing matrix
    val = []
    row_nr = []
    col_nr = []

    # Go through animal herds
    for herd in self.herds:
        sp = herd.species
        br = herd.breed
        ps = herd.prod_system
        ss = herd.sub_system

        if sp != "cattle":
            continue

        def get_uniq(col):
            return herd.data_attr.get("production").columns.unique(col)
        
        # Get all animal products that we are concerned with
        aps = set(get_uniq("animal_prod")) & set(RELEVANT_ANIMAL_PRODUCTS)
        opss = get_uniq("prod_system")
        
        for ap, ops in product(aps, opss):
            if (ops, sp, ap) not in row_idx:
                continue
        
            # Get production of animal product (ap) from output production system (ops) per head
            # of defining animal of species (sp) and breed (br) in production system (ps), sub system (ss)
            # and region (re)
            res = (
                herd.data_attr.get("production")
                .loc[:, (ops, slice(None), ap)]
                .sum(axis=1)
            ) * PROTEIN_CONTENTS[ap]
        
            if all(res == 0):
                continue
        
            val.extend(res)
            col_nr.extend([col_idx.get_loc((sp, br, ps, ss, re)) for re in res.index])
            row_nr.extend(np.zeros(len(res)))

    # Aggregate data_coords_pair to ensure that any overlapping values are summed rather than replace each other
    val, (row_nr, col_nr) = aggregate_data_coords_pair(val, row_nr, col_nr)

    # Create Compressed Sparse Column matrix
    return scipy.sparse.coo_array((val, (row_nr, col_nr)), shape=(1, len(col_idx))).tocsc()

def make_protein_mask_crp():
    val_df = (
        crops.data_attr.get('harvest').loc[['Wheat (add)']].reindex(self.x_idx_short["crp"]).fillna(0) * [PROTEIN_CONTENTS[ADDED_WHEAT]]
        + crops.data_attr.get('harvest').loc[['Peas (add)']].reindex(self.x_idx_short["crp"]).fillna(0) * [PROTEIN_CONTENTS[ADDED_PEAS]]
    )
    
    return scipy.sparse.csc_array(np.atleast_2d(val_df.values))

def make_protein_mask():
    A_ani = make_protein_mask_ani()
    A_crp = make_protein_mask_crp()
    A_fds = scipy.sparse.csc_matrix((1, len(self.x_idx_short["fds"])))

    return scipy.sparse.hstack([A_ani, A_crp, A_fds], format="csc")

make_protein_mask()

<Compressed Sparse Column sparse array of dtype 'float64'
	with 613 stored elements and shape (1, 328637)>

### Quick validity check

Ensure we only have values in the protein map where we expect to, i.e. for the added crops and for cattle.

In [13]:
if scn != "BASELINE":
    df_ani = pd.DataFrame(make_protein_mask_ani(), columns=self.x_idx_short["ani"])
    df_crp = pd.DataFrame(make_protein_mask_crp(), columns=self.x_idx_short["crp"])
    
    # Check that only the added crops have values in the crp part of the mask
    ADDED_CROPS = [ADDED_PEAS, ADDED_WHEAT]
    
    for crop in df_crp.columns.unique("crop"):
        is_all_zeroes = (df_crp.loc[:,(crop, slice(None), slice(None))]==0).all().all()
        assert is_all_zeroes == (crop not in ADDED_CROPS)
    
    # Check that only cattle has values in the ani part of the protein mask
    for sp in df_ani.columns.unique("species"):
        is_all_zeroes = (df_ani.loc[:,(sp, slice(None), slice(None), slice(None), slice(None))]==0).all().all()
        is_cattle = sp == "cattle"
        assert is_all_zeroes != is_cattle

## Construct and replace the `cvxpy.Problem`

In [14]:
prot_mask = make_protein_mask()

def protein_mask_as_opt_goal():
    n = (
        len(self.x_idx_short["ani"])
        + len(self.x_idx_short["crp"])
        + len(self.x_idx_short["fds"])
    )
    x = cvxpy.Variable(n, nonneg=True)

    M = prot_mask
    objective = cvxpy.Maximize(cvxpy.sum(M @ x))

    # Append constraints
    constraints = [
        make_cvxpy_constraint(cons, x) for cons in self.constraints.values()
    ]

    # Define problem
    self.problem = cvxpy.Problem(
        objective=objective,
        constraints=constraints
    )

# Prepare some plot-functions

In [15]:
def mkdirp(filepath: str):
    dirpath = os.path.dirname(filepath)
    if not os.path.exists(dirpath):
        os.mkdir(dirpath)

In [16]:
from matplotlib.colors import TwoSlopeNorm
from matplotlib.ticker import PercentFormatter

def plot_cattle_x0_x_chloropleths(figname: str | None = None):
    x0_cattle = self.x0["ani"].loc[("cattle",)]
    x_cattle = self.x["ani"].loc[("cattle",)].droplevel("sub_system").groupby(["breed", "prod_system", "region"]).sum()
    
    x0_cattle = x0_cattle.unstack("region")
    x_cattle = x_cattle.unstack("region")
    
    assert (x0_cattle.index == x_cattle.index).all(), "Expected index of x0 and x to match"
    
    N = len(x0_cattle.index.values)
    fig, axes = plt.subplots(N, 4, figsize=(20, N * 6))
    
    for pos, axs in zip(x0_cattle.index, axes):
        (ax_l, ax_r, ax_x, ax_x0) = axs
        
        x = x_cattle.loc[pos,:]
        x0 = x0_cattle.loc[pos,:]
    
        # Relative change
        # ---------------
        
        ## Plot 1: Chloropleth (relative)
        x_frac = x / x0
        ### create a normalized colorscheme, with 1=100% in the centre
        vmin, vcenter, vmax, = 0, 1, max(2, x_frac.max())
        norm = TwoSlopeNorm(vmin=vmin, vcenter=vcenter, vmax=vmax)
        ax_l.set_axis_off()
        ax_l.set_title(f"{pos[0]}, {pos[1]}")
        cm.plot.map_from_series(x_frac, ax=ax_l, cmap="RdBu", norm=norm)
        # Set the cbar formatter to percentage 
        ax_l.figure.axes[-1].yaxis.set_major_formatter(PercentFormatter(1))
            
        if any(x_frac > 0):
            ## Plot 2: KDE
            x_frac.plot(kind="kde", ax=ax_r)
            ax_r.xaxis.set_major_formatter(PercentFormatter(1))
            ax_r.set_xlim([0, vmax])
            ax_r.set_title("Distribution of change")
    
        # Absolute values: x and x0
        # -------------------------
        ax_x.set_axis_off()
        ax_x0.set_axis_off()
        x_vmax = max(x.max(), x0.max())
        x_vmin = max(x.min(), x0.min())
        ax_x.set_title("x: New values")
        ax_x0.set_title("x0: Original values")
        cm.plot.map_from_series(x, ax=ax_x, vmin=x_vmin, vmax=x_vmax)
        cm.plot.map_from_series(x0, ax=ax_x0, vmin=x_vmin, vmax=x_vmax)
    
    fig.tight_layout()
    if figname:
        mkdirp(figname)
        fig.savefig(figname)

In [17]:
def plot_crop_x0_x_chloropleths(figname: None | str = None): 
    def get_x(crp: str):
        return self.x["crp"].xs(crp, level="crop", drop_level=False).droplevel("prod_system")
    
    added_peas = get_x("Peas (add)")
    added_wheat = get_x("Wheat (add)")
    crops = [added_peas, added_wheat]
    
    fig, axes = plt.subplots(1, 2, figsize=(8, 7))
    
    for data, ax in zip(crops, axes):
        title = data.index.unique("crop")[0]
        data = data.droplevel("crop")
        ax.set_title(title)
        ax.set_axis_off()
        cm.plot.map_from_series(data, ax=ax)
    
    if figname:
        mkdirp(figname)
        fig.savefig(figname)

# First run: Optimize for protein

In [18]:
%%time
protein_mask_as_opt_goal()
self.solve(
    apply_solution=False,
    verbose=True,
    solver_settings=[{
        "solver": "GUROBI",
        "reoptimize": True,
        "verbose": True,

        #"GURO_PAR_DUMP": 1,

        # Custom params
        "BarConvTol": 1e-3,
        # Sometimes setting Aggregate=0 can improve the model numerics
        "Aggregate": 0,
        
        "NumericFocus": 3,
        # Useful for recognizing infeasibility or unboundedness, but a bit slower than the default algorithm.
        # values: -1 auto, 0 off, 1 force on.
        "BarHomogeneous": 1, 

        # Gurobi has three different heuristic algorithms to find scaling factors. Higher values for the ScaleFlag uses more aggressive heuristics to improve the constraint matrix numerics for the scaled model.
        "ScaleFlag": 2,

        # All constraints must be satisfied to a tolerance of FeasibilityTol.
        # default 1e-6
        "FeasibilityTol": 1e-3,
    }]
)

[09:41:30][FeedDistributor.solve] Finding solution with 'GUROBI' ... ===============================================================================
                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Jan 24 09:41:30 AM: Your problem has 328637 variables, 1604688 constraints, and 0 parameters.
(CVXPY) Jan 24 09:41:30 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 24 09:41:30 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 24 09:41:30 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 24 09:41:30 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                            

RuntimeError: No solution found!

In [19]:
self.problem.variables()[0]

Variable((328637,), var1, nonneg=True)

In [20]:
raise Exception("Stop")

Exception: Stop

In [ ]:
plot_crop_x0_x_chloropleths(f"figures/{scn}/added-crops-distribution-1.png")

In [ ]:
plot_cattle_x0_x_chloropleths(f"figures/{scn}/cattle-x-x0-deviation-1.png")

# Re-run the model to redistribute land-use for minimal change

Now that we optimised for maximum protein, we re-run the model with the protein amount as a constraint, instead optimising for minimal difference from the contemporary configuration.

We thus:
- Extract the max protein value from the previous optimisation run
- Create a constraint that maps x to the protein amount and ensures it is close to the max value
- Reapply the original optimisation goal

In [ ]:
print(f"Max protein calculated as:\n{self.problem.value:e}")

In [ ]:
max_protein_amount = self.problem.value

In [ ]:
def protein_map_as_cons(max_protein_amount = None):
    if max_protein_amount is None:
        raise Exception("Could not get the optimal value from the problem")

    return {
        "left": lambda x, M, b: M @ x - b,
        "right": lambda M, b: 0,
        "rel": ">=",
        "pars": { "M": prot_mask, "b": max_protein_amount }
    }

self.constraints["CX: Protein"] = protein_map_as_cons(0.975 * max_protein_amount)

In [ ]:
self.constraints["CX: Protein"] = protein_map_as_cons(0.90 * 610118100)

In [ ]:
# Overwrite old problem with standard optimization objective,
# but with the protein constraint added
self.problem = self.get_cvx_problem()

In [ ]:
self.solve(
    apply_solution=False,
    verbose=True,
    solver_settings=[{
        "solver": "GUROBI",
        "reoptimize": True,
        "verbose": True,

        
        # Custom params
        # =============
        
        "BarConvTol": 1e-3,
        # Sometimes setting Aggregate=0 can improve the model numerics
        #"Aggregate": 0,
        
        #"NumericFocus": 3,
        # Useful for recognizing infeasibility or unboundedness, but a bit slower than the default algorithm.
        # values: -1 auto, 0 off, 1 force on.
        #"BarHomogeneous": 1, 

        # Gurobi has three different heuristic algorithms to find scaling factors. Higher values for the ScaleFlag uses more aggressive heuristics to improve the constraint matrix numerics for the scaled model.
        #"ScaleFlag": 2,

        # All constraints must be satisfied to a tolerance of FeasibilityTol.
        # default 1e-6
        #"FeasibilityTol": 1e-3,
    }]
)

In [ ]:
plot_crop_x0_x_chloropleths(f"figures/{scn}/added-crops-distribution-2.png")

In [ ]:
plot_cattle_x0_x_chloropleths(f"figures/{scn}/cattle-x-x0-deviation-2.png")

In [ ]:
self.apply_solution()

# Calculate feed s
feed_mgmt.calculate()
# Calculate byprod
byprod_mgmt.calculate()
# Calculate manure
manure_mgmt.calculate()
# Calculate harvest of crop residues
crop_residue_mgmt.calculate()
# Calculate treatment of wastes and other feedstocks
waste.calculate()
# Calculate plant nutrient management
plant_nutrient_mgmt.calculate()
# Calculate energy requirements
machinery_and_energy_mgmt.calculate()
# Calculate inputs supply chain emissions
inputs.calculate()

# Store results

In [ ]:
session.store(
    scn, 2020,
    demand, regions, crops, herds, waste, optproblem
)

In [ ]:
cm.utils.helpers.check_constraints(self)